In [24]:
 !apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
#Pip install required

!pip install datasets
!pip install transformers datasets evaluate
!pip install --upgrade torch torchvision torchaudio
!python -m spacy download en_core_web_lg
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [3]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import csv
from sklearn.manifold import TSNE
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

nlp = spacy.load('en_core_web_lg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from huggingface_hub import notebook_login
notebook_login()


In [5]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
import evaluate
import torch

In [16]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report

Deployment to Hugging Face

In [6]:
# reLoad dataset
dataset = pd.read_csv("/content/yelp_cleaned_sample.csv")

# Sample 1% of train and test sets for faster runtime
train_sample = dataset.sample(frac=0.01, random_state=42)
test_sample = dataset.drop(train_sample.index).sample(frac=0.01, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_sample)
test_dataset = Dataset.from_pandas(test_sample)

# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Define tokenize function
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

# Apply tokenizer
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Rename column
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

# Set format for PyTorch
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

# Define model
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

In [10]:
#Map original labels to binary (0-1 Negative, 3-4 Positive)
def map_to_binary(label):
    if label in [0, 1]:
        return 0
    elif label in [3, 4]:
        return 1
    else:
        return None

dataset['binary_label'] = dataset['label'].apply(map_to_binary)
dataset = dataset.dropna(subset=['binary_label'])
dataset['binary_label'] = dataset['binary_label'].astype(int)

# Data split
train_df, test_df = train_test_split(dataset, test_size=0.2, stratify=dataset['binary_label'], random_state=42)

#Convert to Hugging face
train_dataset = Dataset.from_pandas(train_df[['clean_text', 'binary_label']])
test_dataset = Dataset.from_pandas(test_df[['clean_text', 'binary_label']])

#Relabel
train_dataset = train_dataset.rename_column("binary_label", "labels")
test_dataset = test_dataset.rename_column("binary_label", "labels")

# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(examples):
    return tokenizer(examples['clean_text'], padding='max_length', truncation=True)
# Tokenize and format
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

#Import to torchy
train_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])


# Creat Model
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

#Define Metrics

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    accuracy = accuracy_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": accuracy, "recall": recall, "f1": f1}

#Training Args

repo_name = "RL-LLM"
training_args = TrainingArguments(
    output_dir=repo_name,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=2,
    push_to_hub=True,
)

#Trainer Set-up
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Map:   0%|          | 0/6416 [00:00<?, ? examples/s]

Map:   0%|          | 0/1604 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
#Run Trainer
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,0.221700,0.265552,0.902743,0.889862,0.901141
2,0.074000,0.294440,0.907731,0.898623,0.906566


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=802, training_loss=0.18486880661543467, metrics={'train_runtime': 192.1551, 'train_samples_per_second': 66.779, 'train_steps_per_second': 4.174, 'total_flos': 1718896964960256.0, 'train_loss': 0.18486880661543467, 'epoch': 2.0})

In [17]:
#evaluate results:

# Reults
eval_result = trainer.evaluate()
print("\n Evaluation Results:", eval_result)

#Prediction
preds = trainer.predict(test_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

#Report
print("\n Classification Report:")
print(classification_report(y_true, y_pred, digits=4))


 Evaluation Results: {'eval_loss': 0.29444003105163574, 'eval_accuracy': 0.9077306733167082, 'eval_recall': 0.8986232790988736, 'eval_f1': 0.9065656565656566, 'eval_runtime': 8.1292, 'eval_samples_per_second': 197.314, 'eval_steps_per_second': 24.726, 'epoch': 2.0}

 Classification Report:
              precision    recall  f1-score   support

           0     0.9011    0.9168    0.9089       805
           1     0.9146    0.8986    0.9066       799

    accuracy                         0.9077      1604
   macro avg     0.9079    0.9077    0.9077      1604
weighted avg     0.9078    0.9077    0.9077      1604



In [18]:
trainer.evaluate()

{'eval_loss': 0.29444003105163574,
 'eval_accuracy': 0.9077306733167082,
 'eval_recall': 0.8986232790988736,
 'eval_f1': 0.9065656565656566,
 'eval_runtime': 5.8046,
 'eval_samples_per_second': 276.332,
 'eval_steps_per_second': 34.628,
 'epoch': 2.0}

In [19]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/helpiamlost123/RL-LLM/commit/b3b807971cf0473b5a3b4a8b111194884044cf57', commit_message='End of training', commit_description='', oid='b3b807971cf0473b5a3b4a8b111194884044cf57', pr_url=None, repo_url=RepoUrl('https://huggingface.co/helpiamlost123/RL-LLM', endpoint='https://huggingface.co', repo_type='model', repo_id='helpiamlost123/RL-LLM'), pr_revision=None, pr_num=None)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="helpiamlost123/RL-LLM")